# Capítulo 1: Electrostática y Campos

## Visualizando la Ley de Coulomb
La ecuación clásica es $\vec{E} = k \frac{q}{r^2}\hat{r}$, pero aquí vamos a ver cómo se comportan las líneas de campo realmente.

A continuación, tienes una simulación en vivo. **Prueba cambiar el valor de `q2` a `1.0` para ver la repulsión.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- PARÁMETROS QUE EL ESTUDIANTE PUEDE MODIFICAR ---
# Carga 1 (en el origen)
q1 = 1.0   # Carga positiva
x1, y1 = -1.0, 0.0

# Carga 2 (puedes cambiarla a positiva para ver repulsión)
q2 = -1.0  # Carga negativa (Dipolo)
x2, y2 = 1.0, 0.0
# ----------------------------------------------------

def campo_electrico(x, y, q, x_c, y_c):
    k = 8.99e9
    # Distancia en componentes
    dx = x - x_c
    dy = y - y_c
    # Distancia al cuadrado y al cubo (para el vector unitario)
    r2 = dx**2 + dy**2
    r3 = r2**(1.5)
    
    # Evitar división por cero (pequeño truco numérico)
    r3[r3 == 0] = 1e-10
    
    # Componentes del campo E = k * q / r^2 * (r_vec / r) = k * q * r_vec / r^3
    Ex = k * q * dx / r3
    Ey = k * q * dy / r3
    return Ex, Ey

# Crear una malla de puntos (el "espacio")
x = np.linspace(-3, 3, 100)
y = np.linspace(-2, 2, 100)
X, Y = np.meshgrid(x, y)

# Calcular campo total (Principio de Superposición)
Ex1, Ey1 = campo_electrico(X, Y, q1, x1, y1)
Ex2, Ey2 = campo_electrico(X, Y, q2, x2, y2)

Ex_total = Ex1 + Ex2
Ey_total = Ey1 + Ey2
E_magnitud = np.sqrt(Ex_total**2 + Ey_total**2)

# --- GRAFICAR (La parte visual "Rupturista") ---
plt.figure(figsize=(10, 6))

# Mapa de color logarítmico para la intensidad (para ver el decaimiento)
# Usamos log porque cerca de la carga el campo tiende a infinito
plt.pcolormesh(X, Y, np.log10(E_magnitud), cmap='inferno', shading='auto', alpha=0.3)

# Líneas de campo (Streamplot)
plt.streamplot(X, Y, Ex_total, Ey_total, color='k', linewidth=1, density=1.5, arrowstyle='->')

# Dibujar las cargas
plt.scatter([x1, x2], [y1, y2], c=['red' if q1>0 else 'blue', 'red' if q2>0 else 'blue'], s=200, zorder=10, edgecolors='black')
plt.text(x1, y1+0.15, f"$q_1={q1}$", ha='center', fontsize=12, fontweight='bold')
plt.text(x2, y2+0.15, f"$q_2={q2}$", ha='center', fontsize=12, fontweight='bold')

plt.title(r'Campo Eléctrico Total: $\vec{E} = \vec{E}_1 + \vec{E}_2$', fontsize=14)
plt.xlabel('Distancia x (m)')
plt.ylabel('Distancia y (m)')
plt.xlim(-3, 3)
plt.ylim(-2, 2)
plt.grid(True, linestyle='--', alpha=0.6)

plt.show()

: 

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff
from IPython.display import HTML

# --- 1. CONFIGURACIÓN DEL ESPACIO (Malla de puntos) ---
# Usamos menos puntos que en el mapa de calor para que las flechas no se amontonen
N = 25
x_range = np.linspace(-3, 3, N)
y_range = np.linspace(-2, 2, N)
X, Y = np.meshgrid(x_range, y_range)

# Posiciones de las cargas
x1, y1 = -1.0, 0.0 # Carga fija Q1
x2, y2 = 1.0, 0.0  # Carga variable Q2

# --- 2. FUNCIÓN DE CÁLCULO VECTORIAL ---
def calcular_vectores_campo(q_variable):
    k = 8.99e9
    epsilon = 0.1 # Factor de suavizado para evitar flechas infinitas cerca de las cargas

    # Vector r1 (desde Q1 hasta cada punto de la malla)
    dx1 = X - x1
    dy1 = Y - y1
    r1_sq = dx1**2 + dy1**2 + epsilon**2 # Distancia suavizada al cuadrado
    r1 = np.sqrt(r1_sq)
    # Campo E1 (Componentes vectoriales E = k*q*vec_r / r^3)
    Ex1 = k * 1.0 * dx1 / (r1**3)
    Ey1 = k * 1.0 * dy1 / (r1**3)

    # Vector r2 (desde Q2 variable)
    dx2 = X - x2
    dy2 = Y - y2
    r2_sq = dx2**2 + dy2**2 + epsilon**2
    r2 = np.sqrt(r2_sq)
    # Campo E2
    Ex2 = k * q_variable * dx2 / (r2**3)
    Ey2 = k * q_variable * dy2 / (r2**3)

    # Principio de Superposición (Suma vectorial)
    Ex_total = Ex1 + Ex2
    Ey_total = Ey1 + Ey2
    
    return Ex_total, Ey_total

# --- 3. GENERACIÓN DE FRAMES (VISUALIZACIÓN) ---
# Valores de carga Q2 para el deslizador
valores_q2 = np.linspace(-5, 5, 21) # 21 pasos para una animación fluida

# --- Función auxiliar para crear la figura de flechas ---
def crear_figura_quiver(q_val):
    Ex, Ey = calcular_vectores_campo(q_val)
    # Usamos figure_factory para crear el mapa de flechas (Quiver)
    # scale: ajusta la longitud de las flechas
    # arrow_scale: ajusta el tamaño de la cabeza de la flecha
    fig_quiver = ff.create_quiver(X, Y, Ex, Ey, scale=0.02, arrow_scale=0.3, name='Líneas de Campo', line=dict(color='black', width=1))
    return fig_quiver

# Creamos la FIGURA BASE (Estado inicial)
fig_base = crear_figura_quiver(valores_q2[0])
fig = go.Figure(data=fig_base.data)

# Creamos los FRAMES de animación
frames = []
for q_val in valores_q2:
    # Generamos una figura temporal para este valor de Q
    temp_fig = crear_figura_quiver(q_val)
    # Extraemos los datos de las líneas de esa figura y creamos un frame
    frames.append(go.Frame(data=temp_fig.data, name=f"q={q_val:.1f}"))

fig.frames = frames

# --- 4. CONFIGURACIÓN DEL LAYOUT (Deslizador y Cargas) ---
# Añadir marcadores para las cargas
fig.add_trace(go.Scatter(x=[x1], y=[y1], mode='markers+text', marker=dict(size=18, color='red', line=dict(width=2, color='black')), name='Q1 (+1C)', text=['+'], textposition='middle center', textfont=dict(color='white', size=14)))
fig.add_trace(go.Scatter(x=[x2], y=[y2], mode='markers+text', marker=dict(size=18, color='blue', line=dict(width=2, color='black')), name='Q2 (Variable)', text=['Q2'], textposition='top center'))

# Configurar el deslizador
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Carga Q2: ", "suffix": " C"},
    pad={"t": 50},
    steps=[dict(
        method='animate',
        # args: [nombre_frame, configuración_animación]
        args=[[f"q={qv:.1f}"], dict(mode='immediate', frame=dict(duration=0, redraw=False), transition=dict(duration=0))],
        label=f"{qv:.1f}"
    ) for qv in valores_q2]
)]

fig.update_layout(
    title="Dirección del Campo Eléctrico (Diagrama Vectorial)",
    xaxis=dict(title="Distancia X", range=[-3.2, 3.2], zeroline=False),
    yaxis=dict(title="Distancia Y", range=[-2.2, 2.2], zeroline=False),
    sliders=sliders,
    width=750, height=550,
    showlegend=False,
    plot_bgcolor='rgba(240,240,240,0.5)' # Fondo gris claro
)

# --- 5. INYECCIÓN HTML PARA LA WEB ---
HTML(fig.to_html(include_plotlyjs='cdn'))

In [ ]:
# --- CÓDIGO PARA VISUALIZACIÓN 3D DEL POTENCIAL ---
# Este gráfico muestra el voltaje como una altura (Eje Z)

# 1. FUNCIÓN DE POTENCIAL (Escalar)
def calcular_potencial(q_variable):
    k = 8.99e9
    
    # Potencial V1 (Carga fija)
    dx1, dy1 = X - x1, Y - y1
    r1 = np.sqrt(dx1**2 + dy1**2)
    # Evitamos la singularidad (división por cero) poniendo un tope mínimo
    r1[r1 < 0.2] = 0.2 
    V1 = k * q1 / r1
    
    # Potencial V2 (Carga variable)
    dx2, dy2 = X - x2, Y - y2
    r2 = np.sqrt(dx2**2 + dy2**2)
    r2[r2 < 0.2] = 0.2
    V2 = k * q_variable / r2
    
    V_total = V1 + V2
    
    # Cortamos los picos muy altos para que el gráfico se vea bonito
    # (Saturamos el voltaje entre -5e10 y 5e10)
    V_total = np.clip(V_total, -5e10, 5e10)
    
    return V_total

# 2. CREAR FIGURA 3D
fig_3d = go.Figure()

# Estado inicial (q = -5, el primer valor del array valores_q2)
# Usamos Surface para crear "terreno"
fig_3d.add_trace(go.Surface(
    z=calcular_potencial(valores_q2[0]),
    x=X,
    y=Y,
    colorscale='Viridis',
    name='Potencial V'
))

# 3. CREAR FRAMES DE ANIMACIÓN 3D
frames_3d = []
for q_val in valores_q2:
    frames_3d.append(go.Frame(
        data=[go.Surface(z=calcular_potencial(q_val))],
        name=f"q={q_val:.1f}"
    ))

fig_3d.frames = frames_3d

# 4. CONFIGURAR SLIDER (Mismo mecanismo que el anterior)
sliders_3d = [dict(
    active=0,
    currentvalue={"prefix": "Carga Q2: ", "suffix": " C"},
    pad={"t": 50},
    steps=[dict(
        method='animate',
        args=[[f"q={qv:.1f}"], dict(mode='immediate', frame=dict(duration=0, redraw=True), transition=dict(duration=0))],
        label=f"{qv:.1f}"
    ) for qv in valores_q2]
)]

fig_3d.update_layout(
    title="Topografía del Potencial Eléctrico (3D)",
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Voltaje (V)',
        zaxis=dict(range=[-5e10, 5e10]), # Fijamos la altura para que no salte
        aspectratio=dict(x=1, y=1, z=0.7) # Proporción del cubo 3D
    ),
    sliders=sliders_3d,
    width=800, height=600,
    margin=dict(l=0, r=0, b=0, t=50) # Márgenes estrechos para aprovechar espacio
)

# 5. INYECCIÓN HTML
HTML(fig_3d.to_html(include_plotlyjs='cdn'))

### Electrostática

Carga eléctrica $(q)$ es una propiedad intrínseca.

* Protones $(q>0)$
* Electrones $(q<0)$
* Neutrones $(q=0)$

La unidad de carga eléctrica es el Coulomb.

La carga elemental es $e = 1{,}6 \times 10^{-19}$ C.

* Protones $q_p=+e$
* Electrones $q_e = -e$

Un cuerpo puede ceder o ganar electrones, lo que le da una carga positiva o negativa.

### Ley de Coulomb

* Existen dos tipos de cargas: positivas y negativas.
* Dos cargas interactúan a distancia con una fuerza de magnitud inversamente proporcional al cuadrado de la distancia que las separa.
* Esta fuerza es proporcional al producto de las cargas.

La magnitud de la fuerza es:

$$F = k \dfrac{|q_1 q_2|}{d^2} = \dfrac{1}{4\pi\epsilon_0}\dfrac{|q_1 q_2|}{d^2}$$

donde:
* $k =  8{,}98755 \times 10^9 \ \dfrac{\text{Nm}^2}{\text{C}^2} \approx 9 \times 10^9 \ \dfrac{\text{Nm}^2}{\text{C}^2}$
* $\epsilon_0$ es la permitividad en el vacío, con  $\epsilon_0 = 8{,}854 \times 10^{-12} \ \dfrac{\text{C}^2}{\text{Nm}^2}$.

#### Forma Vectorial General

Si consideramos el origen $O$ y vectores de posición:

$$
\begin{align*}
\vec{F}_{12} &= \dfrac{1}{4\pi\epsilon_0} \ \dfrac{q_1 q_2}{d^2} \ \hat{u}_{F} \\
&= \frac{1}{4\pi\epsilon_0} \ \frac{q_1 q_2}{|\vec{r}_1 - \vec{r}_2|^2} \ \hat{u}_F \\
&= \frac{1}{4\pi\epsilon_0} \ \frac{q_1 q_2}{|\vec{r}_1 - \vec{r}_2|^2} \ \frac{(\vec{r}_1 - \vec{r}_2)}{|\vec{r}_1 - \vec{r}_2|} \\
\vec{F}_{12} &= \frac{1}{4\pi\epsilon_0} \ q_1 q_2 \ \frac{(\vec{r}_1 - \vec{r}_2)}{|\vec{r}_1 - \vec{r}_2|^3}
\end{align*}
$$

Si hay $N$ cargas, la fuerza total que siente la carga $q_1$ está dada por:

$$\vec{F}_1 = \frac{q_1}{4\pi\epsilon_0} \sum_{j=2}^{N} q_j \frac{\vec{r}_1 - \vec{r}_j}{|\vec{r}_1 - \vec{r}_j|^3}$$

Este análisis es para distribuciones discretas (cargas puntuales).

```{image} images_tikz/01.svg
:alt: Diagrama vectorial de la Ley de Coulomb
:width: 80%
:align: center

<div align="center" style="margin: 20px 0;">
    <h3>Laboratorio Virtual: Cargas y Campos</h3>
    <p><i>Instrucciones: Arrastra cargas rojas y azules al centro para ver cómo cambia el campo.</i></p>
    
    <iframe src="https://phet.colorado.edu/sims/html/charges-and-fields/latest/charges-and-fields_es.html"
            width="800"
            height="600"
            scrolling="no"
            allowfullscreen>
    </iframe>
    
</div>

<div align="center" style="margin: 20px 0;">
    <h3>Simulación GeoGebra: Ley de Coulomb</h3>
    
    <iframe src="https://www.geogebra.org/material/iframe/id/b8gq9p9r/width/800/height/500/border/888888/sfsb/true/smb/false/stb/false/stbh/false/ai/false/asb/false/sri/true/rc/false/ld/false/sdz/true/ctl/false"
            width="100%"
            height="500px"
            style="border: 1px solid #e4e4e4; border-radius: 4px;"
            allowfullscreen>
    </iframe>
    
</div>

<div style="position: relative; width: 100%; height: 0; padding-bottom: 75%; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">
    <iframe src="https://phet.colorado.edu/sims/html/coulombs-law/latest/coulombs-law_es.html"
            style="position: absolute; top: 0; left: 0; width: 100%; height: 100%; border: none;"
            allowfullscreen>
    </iframe>
</div>
<p align="center" style="font-size: 0.9em; color: gray;">Simulación cortesía de PhET Interactive Simulations</p>